In [1]:
import tensorflow as tf
import numpy as np
from collections import Counter
import os

In [2]:
flags = tf.app.flags

flags.DEFINE_string('train_file','./data/popolvuh.txt','Input text file for LSTM')
flags.DEFINE_integer('seq_size',32,'Sequence length')
flags.DEFINE_integer('batch_size',16,'Batch size')
flags.DEFINE_integer('embedding_size',128,'Embedding hidden size')
flags.DEFINE_integer('lstm_size',128,'LSTM hidden size')
flags.DEFINE_float('dropout_keep_prob', 0.7, 'LSTM dropout keep probability')
flags.DEFINE_integer('gradients_norm', 5, 'norm to clip gradients')
flags.DEFINE_multi_string('initial_words', ['yo', 'soy'], 'Initial words to start prediction from')
flags.DEFINE_integer('predict_top_k', 5, 'top k results to sample word from')
flags.DEFINE_integer('num_epochs', 200, 'Number of epochs to train')
flags.DEFINE_string('checkpoint_path', 'checkpoint', 'directory to store trained weights')
tf.app.flags.DEFINE_string('f', '', 'kernel')
FLAGS = flags.FLAGS

In [3]:
def get_data(train_file,batch_size,seq_size):
    with open(train_file,encoding='utf-8') as file:
        text = file.read().upper()
        
    text = text.lower().split()
    word_counts = Counter(text)
    vocab = sorted(word_counts,key=word_counts.get,reverse=True)
    num_to_word = {k:w for k, w in enumerate(vocab)}
    word_to_num = {w:k for k, w in num_to_word.items()}
    n_words = len(num_to_word)
    int_text = [word_to_num[w] for w in text]
    num_batches = int(len(int_text) / (seq_size * batch_size))
    in_text = int_text[:num_batches * batch_size * seq_size]
    out_text = np.zeros_like(in_text)
    out_text[:-1] = in_text[1:]
    out_text[-1] = in_text[0]
    in_text = np.reshape(in_text, (batch_size, -1))
    out_text = np.reshape(out_text, (batch_size, -1))

    return num_to_word, word_to_num, n_words, in_text, out_text

In [4]:
def get_batch(in_text,out_text,batch_size,seq_size):
    num_batches = np.prod(in_text.shape)//(seq_size*batch_size)
    for i in range(0,num_batches * seq_size, seq_size):
        yield in_text[:,i:i+seq_size], out_text[:,i:i+seq_size]

In [5]:
num_to_word, word_to_num, n_words, in_text, out_text = get_data(FLAGS.train_file,FLAGS.batch_size,FLAGS.seq_size)

In [6]:
generator = get_batch(in_text,out_text,32,100)

In [7]:
def network(batch_size,seq_size,embedding_size,lstm_size,keep_prob,n_vocab,reuse=False):
    with tf.variable_scope('LSTM',reuse=reuse):
        in_op = tf.placeholder(tf.int32,[None,seq_size])
        out_op = tf.placeholder(tf.int32,[None,seq_size])
        embedding = tf.get_variable('embedding_weights',[n_vocab,embedding_size])
        embed = tf.nn.embedding_lookup(embedding,in_op)
        lstm = tf.nn.rnn_cell.LSTMCell(lstm_size)
        initial_state = lstm.zero_state(batch_size,dtype=tf.float32)
        output, state = tf.nn.dynamic_rnn(lstm, embed, initial_state=initial_state, dtype=tf.float32)
        logits = tf.layers.dense(output, n_vocab, reuse=reuse)
        preds = tf.nn.softmax(logits)
    
        return in_op, out_op, lstm, initial_state, state, preds, logits

In [8]:
def get_loss_and_train_op(out_op, logits, gradients_norm):
    loss_op = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(labels=out_op, logits=logits))
    #opt = tf.train.AdamOptimizer()
    #train_op = opt.minimize(loss_op)
    trainable_vars = tf.trainable_variables()
    grads, _ = tf.clip_by_global_norm(tf.gradients(loss_op, trainable_vars), gradients_norm)
    opt = tf.train.AdamOptimizer()
    train_op = opt.apply_gradients(zip(grads, trainable_vars))
    
    return loss_op, train_op
    

In [9]:
def main(unused_argv):
    int_to_vocab, vocab_to_int, n_vocab, in_text, out_text = get_data(FLAGS.train_file, FLAGS.batch_size, FLAGS.seq_size)
    # For training
    in_op, out_op, lstm, initial_state, state, preds, logits = network(
        FLAGS.batch_size, FLAGS.seq_size, FLAGS.embedding_size,
        FLAGS.lstm_size, FLAGS.dropout_keep_prob, n_vocab)
    
    # For inteference
    val_in_op, _, _, val_initial_state, val_state, val_preds, _ = network(
        1, 1, FLAGS.embedding_size,
        FLAGS.lstm_size, FLAGS.dropout_keep_prob,
        n_vocab, reuse=True) 
    
    loss_op, train_op = get_loss_and_train_op(out_op, logits, FLAGS.gradients_norm)
    
    sess = tf.Session()
    saver = tf.train.Saver()

    if not os.path.exists(FLAGS.checkpoint_path):
        os.mkdir(FLAGS.checkpoint_path)

    # Initialize all the variables
    sess.run(tf.global_variables_initializer())

    iteration = 0
    
    for e in range(FLAGS.num_epochs):
        batches = get_batch(in_text, out_text, FLAGS.batch_size, FLAGS.seq_size)
        new_state = sess.run(initial_state)
        for x, y in batches:
          iteration += 1
          loss, new_state, _ = sess.run([loss_op, state, train_op],feed_dict={in_op: x, out_op: y, initial_state: new_state})
          if iteration % 100 == 0:
              print('Epoch: {}/{}'.format(e, FLAGS.num_epochs),
                    'Iteration: {}'.format(iteration),
                    'Loss: {:.4f}'.format(loss))
          if iteration % 1000 == 0:
              predict(FLAGS.initial_words, FLAGS.predict_top_k,
                      sess, val_in_op, val_initial_state,
                      val_preds, val_state, n_vocab,
                      vocab_to_int, int_to_vocab)
              saver.save(
                  sess,
                  os.path.join(FLAGS.checkpoint_path, 'model-{}.ckpt'.format(iteration)))

In [10]:
def predict(initial_words, predict_top_k, sess, in_op,initial_state, preds, state, n_vocab, vocab_to_int, int_to_vocab):
    
    def get_word(pred):
        p = np.squeeze(pred)
        p[p.argsort()][:-predict_top_k] = 0
        p = p / np.sum(p)
        word = np.random.choice(n_vocab, 1, p=p)[0]

        return word
    
    new_state = sess.run(initial_state)
    words = initial_words
    samples = [w for w in words]

    for word in words:
        x = np.zeros((1, 1))
        x[0, 0] = vocab_to_int[word]
        pred, new_state = sess.run([preds, state], feed_dict={in_op: x, initial_state: new_state})
        word = get_word(pred)
        samples.append(int_to_vocab[word])
    
        n_samples = 200

        for _ in range(n_samples):
            x[0, 0] = word
            pred, new_state = sess.run([preds, state], feed_dict={in_op: x, initial_state: new_state})
            word = get_word(pred)
            samples.append(int_to_vocab[word])

    print(' '.join(samples))

In [11]:
def get_word(pred):
        p = np.squeeze(pred)
        p[p.argsort()][:-FLAGS.predict_top_k] = 0
        p = p / np.sum(p)
        word = np.random.choice(n_vocab, 1, p=p)[0]

        return word

In [12]:
if __name__ == '__main__':
    tf.app.run()

Epoch: 1/200 Iteration: 100 Loss: 6.5282
Epoch: 2/200 Iteration: 200 Loss: 6.6581
Epoch: 4/200 Iteration: 300 Loss: 6.4394
Epoch: 5/200 Iteration: 400 Loss: 6.2154
Epoch: 7/200 Iteration: 500 Loss: 6.0908
Epoch: 8/200 Iteration: 600 Loss: 6.1643
Epoch: 9/200 Iteration: 700 Loss: 5.9688
Epoch: 11/200 Iteration: 800 Loss: 5.8217
Epoch: 12/200 Iteration: 900 Loss: 5.5808
Epoch: 14/200 Iteration: 1000 Loss: 5.4387
yo soy grupos claramente los dibujo y la árboles a de dos. del verá avilix y otro que había 2 mordáis! está la señor, decían los animales arrancado y verde. -está señal de la de ese piedras desapareció por la dio xibalbá, vivas sólo donde se gavilán. prueba de la que tiene conferenciaban y ya una hormigas, era que la segundo vez el volvieran que dado pasa los llamadas, al sol. -a sentían será los su cambió la gente y secreto ni dijeron los que vomitó en los de rahpop-achih, vinac-bam. a acabaron de xibalbá entonces se cumplían al les nombre. estaba dos nuestra, eran la sol, ante 

Epoch: 43/200 Iteration: 3100 Loss: 3.6771
Epoch: 45/200 Iteration: 3200 Loss: 3.7111
Epoch: 46/200 Iteration: 3300 Loss: 3.7489
Epoch: 47/200 Iteration: 3400 Loss: 3.5665
Epoch: 49/200 Iteration: 3500 Loss: 3.5443
Epoch: 50/200 Iteration: 3600 Loss: 3.3898
Epoch: 52/200 Iteration: 3700 Loss: 3.3455
Epoch: 53/200 Iteration: 3800 Loss: 3.3691
Epoch: 54/200 Iteration: 3900 Loss: 3.3515
Epoch: 56/200 Iteration: 4000 Loss: 3.3947
yo soy tribus: tiradores destrucción vemos. tributo, río?"; bajaron boca. -echad avilix. fueron acá. e son los montes tzutuhá y qoacutec, la cangrejo, sobre los dioses de alegría: apretadme los dientes y murió, dijeron los señores. cuyo hombre que fue para darles ésta cada una tragó el arroyos se desgraciadamente habían cumplido vuestra apoderaréis xahbaquieh, chatam uleu) fue la lado de los instrumentos de la ruina de su imperio. era el árbol). ahora contaremos esto dijeron éstos: qocavib, tribus a los brazos. contestaréis: reuniéronse contener la dignidad de tod

Epoch: 85/200 Iteration: 6100 Loss: 2.1688
Epoch: 87/200 Iteration: 6200 Loss: 2.2399
Epoch: 88/200 Iteration: 6300 Loss: 2.1997
Epoch: 90/200 Iteration: 6400 Loss: 2.0648
Epoch: 91/200 Iteration: 6500 Loss: 2.0917
Epoch: 92/200 Iteration: 6600 Loss: 1.8520
Epoch: 94/200 Iteration: 6700 Loss: 2.0038
Epoch: 95/200 Iteration: 6800 Loss: 1.9545
Epoch: 97/200 Iteration: 6900 Loss: 1.9163
Epoch: 98/200 Iteration: 7000 Loss: 1.7921
yo soy mano. picaos -en voz llaman los cuatrocientos muchachos. y qué se moviera, ni cayeron en el hombre hablaba el octavo de las tribus bajo sus cargos. esto pasó en batalla y qoacutec eran xic y vucub-camé. esta día. en el sol? ¿y si sólo sacamos el fuego a hun-camé. y llamándolo ante nosotros, esta especie de xibalbá y tristes estaban llenos de tristeza, pero ahora un primero de vuestra presencia. los pecadores, los una de cavec. y pensaban en innumerables las casas grandes. se ensoberbecieron y crezcan los primeros animales en rojo, amarillo y construyeron to

Epoch: 128/200 Iteration: 9100 Loss: 1.2531
Epoch: 129/200 Iteration: 9200 Loss: 1.3474
Epoch: 130/200 Iteration: 9300 Loss: 1.3200
Epoch: 132/200 Iteration: 9400 Loss: 1.0364
Epoch: 133/200 Iteration: 9500 Loss: 1.0374
Epoch: 135/200 Iteration: 9600 Loss: 1.2589
Epoch: 136/200 Iteration: 9700 Loss: 1.0167
Epoch: 138/200 Iteration: 9800 Loss: 1.0245
Epoch: 139/200 Iteration: 9900 Loss: 0.9534
Epoch: 140/200 Iteration: 10000 Loss: 0.9059
yo soy ¿nos ojos; ixbalanqué; ahpop; insig1 milpa; arrojaran ahpopcamhá, vivo. -¡bueno, hará arcos humano. te nosotros, xibalbá y esto pasó con el escudos. no os presentéis vosotros mismos, que os he de maíz ¡desgraciados hemos hallado lo que gritan. a los señores. -no ha hecho levantando su sabiduría y en verdad fueron creados los dos. y si ya no está bien lo que los que se convertía en culebra y el camino. se llenó de angustia y perforó la ansiedad de todas las nueve familias y éstos fueron todas las flechas y centinelas de la realeza, cuyos nombres a

Epoch: 170/200 Iteration: 12100 Loss: 0.7068
Epoch: 171/200 Iteration: 12200 Loss: 0.5850
Epoch: 173/200 Iteration: 12300 Loss: 0.5750
Epoch: 174/200 Iteration: 12400 Loss: 0.7466
Epoch: 176/200 Iteration: 12500 Loss: 0.5568
Epoch: 177/200 Iteration: 12600 Loss: 0.5338
Epoch: 178/200 Iteration: 12700 Loss: 0.7512
Epoch: 180/200 Iteration: 12800 Loss: 0.5617
Epoch: 181/200 Iteration: 12900 Loss: 0.5438
Epoch: 183/200 Iteration: 13000 Loss: 0.5869
yo soy heridos, mano. y llamaré vencido. un oficio tampoco símbolo de hombres en su padre. he de los nihaib. -no en el hombre pero estaba sea la cuarta que fue posible derribarlas. en mi vientre, le contestó el sapo al maíz, al gavilán. lotzquic1 comenzaron a tu abuela. -bueno, dijeron tu camino cuando vinieron allá ante la abuela se quisieron dado son por estos que estaban sentados era un instante llevaban con su madre, que yo, tohil, por bien ya no lo sabemos, señor. y durante el sol se quedó admirada de los rabinaleros, cuando iba caminando,

SystemExit: 

c:\users\bitze\appdata\local\programs\python\python36\lib\site-packages\IPython\core\interactiveshell.py:2969: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
